In [1]:
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.types import AgentMessage
from aap_core.agent import BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import ParallelAgent
from aap_core.types import AgentResponse
from aap_dspy.chain import BaseSignatureAdapter, ChatCausalMultiTurnsChain

import dspy

In [2]:
class QA(dspy.Signature):
    guide: str = dspy.InputField(optional=True)
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()
    history: dspy.History = dspy.InputField()


class SignatureAdapter(BaseSignatureAdapter[QA]):
    def msg2sig(self, message: AgentMessage) -> List[QA]:
        history = dspy.History(messages=[])
        q, a = None, None
        for response in message.responses:
            if response[0] == "user":
                q = response[1]
            elif response[0] == "tool":
                pass
            elif response[0] == "system":
                pass
            else:
                a = response[1]

            if q is not None and a is not None:
                history.messages.append({"question": q, "answer": a})
                q, a = None, None

        guide_str = self._prefill_dict.get("guide", "")
        qa = QA(question=message.query, answer="", history=history, guide=guide_str)
        return [qa]

    def sig2msg(self, signatures: List[QA], name: str) -> List[AgentResponse]:
        responses = []
        for signature in signatures:
            responses.append(("user", signature.question))
            responses.append((name, signature.answer))
        return responses

In [3]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [4]:
system_prompt = """You are a genius and creative writer."""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")
agent1_skill = AgentSkill(
    id='agent1-skill',
    name="agent1 skill",
    description="self-agent1 skill",
    tags=['agent1']
)
agent1_card = AgentCard(
    name="agent1 agent",
    description="self-agent1 agent",
    skills=[agent1_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent2_skill = AgentSkill(
    id='agent2-skill',
    name="agent2 skill",
    description="self-agent2 skill",
    tags=['agent2']
)
agent2_card = AgentCard(
    name="agent2 agent",
    description="self-agent2 agent",
    skills=[agent2_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
parallel_skill = AgentSkill(
    id='parallel-skill',
    name="parallel skill",
    description="self-parallel skill",
    tags=['parallel']
)
parallel_card = AgentCard(
    name="parallel agent",
    description="self-parallel agent",
    skills=[parallel_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
model = dspy.LM(model="ollama_chat/llama3.2:latest", api_base="http://192.168.55.1:11434", api_key="", temperature=1.5, cache=False)
predictor = dspy.ChainOfThought(QA)
adapter = SignatureAdapter.with_prefill({"guide": system_prompt})
chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=adapter).with_lm(model)
agent1 = Agent(chain=chain, card=agent1_card, state_change_callback=state_callback)
agent2 = Agent(chain=chain, card=agent2_card, state_change_callback=state_callback)
parallel_agent = ParallelAgent(agents=[agent1, agent2], card=parallel_card, state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query))

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:running))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:idle/((agent1 agent:idle)|(agent2 agent:idle))
query='' query_media=None origin='parallel agent' responses=[('agent2 agent', "When thou art gone, my heart doth ache in pain,\nMy love for thee doth rise like Phoebe's rain.\nLove's gentle touch can set thy soul aflame,\nIn love's sweet bonds, our hearts forever entwine."), ('agent2 agent', "When thou art gone, my heart doth ache in pain,\nMy love for thee doth rise like Phoebe's rain.\nLove's gentle touch can set thy soul aflame,\nIn love's sweet bonds, our hearts forever entwine.")] context=None execution_result='s

In [6]:
dspy.inspect_history(n=10)





[2025-12-10T12:01:56.488082]

System message:

Your input fields are:
1. `guide` (str): 
2. `question` (str): 
3. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
You are a genius and creative writer.

[[ ## question ## ]]
Write a short poem with Shakespere's style about love

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
To craft a poem